In [13]:

import random
import math
import statistics
import numpy as np
def divide_RNG_data_into_10_equal_subdivisions_and_count( data_points ):
    subdivisions = {  "1":  0,
                      "2":  0,
                      "3":  0,
                      "4":  0,
                      "5":  0,
                      "6":  0,
                      "7":  0,
                      "8":  0,
                      "9":  0,
                      "10": 0   }
 

    # Loop through our data points and count number of data points in each subdivision
    # Divide by tenths, from 0.0 to 1.0.
    for num in data_points:
        num = float(num)
        if num < 0.1:
            subdivisions["1"] += 1
        elif num < 0.2:
            subdivisions["2"] += 1
        elif num < 0.3:
            subdivisions["3"] += 1
        elif num < 0.4:
            subdivisions["4"] += 1
        elif num < 0.5:
            subdivisions["5"] += 1
        elif num < 0.6:
            subdivisions["6"] += 1
        elif num < 0.7:
            subdivisions["7"] += 1
        elif num < 0.8:
            subdivisions["8"] += 1
        elif num < 0.9:
            subdivisions["9"] += 1
        elif num < 1.0:
            subdivisions["10"] += 1

    return subdivisions

def chi_sq_significance_test( chi_sq, signif_level):
    result = "FAIL TO REJECT null hypothesis"
    crit_value = 0.0
    if signif_level == 0.8:
        crit_value = 10118.8246
    elif signif_level == 0.90:
        crit_value = 10181.6616
    elif signif_level == 0.95:
        crit_value = 10233.7489
    elif signif_level == 0.05:    
        crit_value = 9768.5251
    else:
        print( "**Invalid Significance Level for Chi Sq***")

    if chi_sq > crit_value:
        result = "REJECT null hypothesis"

    print ("Print Significance Level: " + str(signif_level))
    print ("Chi Sq: " + str(chi_sq))
    print ("Crit Value: " + str(crit_value))
    print ("Result is: " + result)
    print ("....................................")

    return result

def chi_square_uniformity_test( data_set, confidence_level, num_samples ):
    chi_sq_value = 0.0
    degrees_of_freedom = num_samples - 1

    # We're doing 10 equal subdivisions, so need to divide our number samples by 10,
    # Assuming uniform distribution, to get an expected value. All values should be same
    # If our distro is actually uniform.
    expected_val = num_samples/10.0


    # Loop through a dictionary and get every count
    # The observed value is going to be our count at each key, and then we can do chi-square
    for observed_val in data_set:
        # print "Observed value is: " + observed_val
        chi_sq_value += ( pow((expected_val - data_set[observed_val]), 2)/expected_val )

    # Coming out of this loop, we'll have a chi-squared test statistic
    # Now we just need to do a lookup to see if it's valid
    return chi_sq_value

def kolmogorov_smirnov_test( data_set, confidence_level, num_samples ):
    
    # Step 1:  Rank data from smallest to largest, such that:
    # R(1) <= R(2) <= R(3) ... <= R(i)
    data_set.sort()

    # Step 2: Computer D+ and D-
    # D+ = max(i/N - R(i))
    d_plus = get_d_plus_value_for_KS_TEST(data_set, num_samples)
    print ("D+ VALUE ="+str(d_plus))

    # D- = max(R(i) - (i -1)/n)
    d_minus = get_d_minus_value_for_KS_TEST(data_set, num_samples)
    print ("D- VALUE="+str(d_minus))

    # Step 3:  Computer D = max(D+,D-)
    d_value = max(d_plus, d_minus)
    print ("D VALUE (max): "+str(d_value))

    # Step 4: Determine critical value, using table
    # Step 5: Accept or reject Null hypothesis
    return d_value
def collect_first_100_samples_in_data_set( data_file ):
   

    first_100_vals_as_FLOATS = data_file[:100]
    #print("f=-",first_100_vals_as_FLOATS)
    return first_100_vals_as_FLOATS   
def ks_significance_test( d_statistic, num_observations, alpha_level ):
    """
    Perform Significance test for Kolmogorov-Smirnov
    Uses formulas from table A.7:  Discrete-Event System Simulation, by Banks and Carson, 1984
    :param d_statistic: The d-value we are testing
    :param num_observations: The number of observations in our data set
    :param alpha_level: The level of significance we are testing
    :return: result -- accept or reject
    """
    result = "FAIL TO REJECT null hypothesis"
    critical_value = 0


    if alpha_level == 0.1:
        critical_value = 1.22/np.sqrt(num_observations)
    elif alpha_level == 0.05:
        critical_value = 1.36/np.sqrt(num_observations)
    elif alpha_level == 0.01:
        critical_value = 1.63/np.sqrt(num_observations)
    else:
        print ("Invalid alpha level for KS test. Must be: 0.1, 0.05, or 0.01")

    if d_statistic > critical_value:
        result = "REJECT null hypothesis"
    print ("Alpha Level is: " + str(alpha_level))
    print ("D_statistic is: " + str(d_statistic))
    print ("Critical value is: " + str(critical_value))
    print ("Result is: " + result)
    print ("............................")

    return result
def get_d_plus_value_for_KS_TEST( data_set, num_samples ):
    """
    Finds the D+ value for a KS test
    :param data_set: 100 values, must be a list of floats
    :return: the D-+Statistic for our data set
    """
    # D+ = max(i/N - R(i))
    d_plus_max = 0
    value_rank_i = 1

    # iterate through data set
    for value in data_set:
        # Do each D+ calculation, store it
        d_plus_i_value = ( (value_rank_i/num_samples) - value )

        # Check if it is highest D+ value yet
        if d_plus_i_value > d_plus_max:
            d_plus_max = d_plus_i_value

        # increment our "i" value
        value_rank_i = value_rank_i + 1

    # coming out of this loop, D+ = highest D+ value
    return d_plus_max


def get_d_minus_value_for_KS_TEST( data_set, num_samples ):
    """
    Finds the D- value for a KS test
    :param data_set: 100 values, must be a list of floats
    :return: the D- Statistic for our data set
    """
    # D- = max(R(i) - (i -1)/n)
    d_minus_max = 0
    value_rank_i = 1.0

    # iterate through data set
    for value in data_set:
        # Do each D+ calculation, store it
        substraction_value = ( (value_rank_i - 1.0)/num_samples )
        d_minus_i_value = value - substraction_value

        # Check if it is highest D+ value yet
        if d_minus_i_value > d_minus_max:
            d_minus_max = d_minus_i_value

        # increment our "i" value
        value_rank_i = value_rank_i + 1

    # coming out of this loop, D+ = highest D+ value
    return d_minus_max


def linearCongruentialMethod(Xo, m, a, c,
                            randomNums,
                            noOfRandomNums):

    # Initialize the seed state
    randomNums[0] = Xo

    # Traverse to generate required
    # numbers of random numbers
    for i in range(1, noOfRandomNums):

        # Follow the linear congruential method
        randomNums[i] = ((randomNums[i - 1] * a) +
                                    c) % m

# Driver Code
if __name__ == '__main__':
    
    # Seed value
    Xo = 123456789.0
    
    # Modulus parameter
    m = (2 ** 16) 

    # Multiplier term
    a =  101427 
    
    # Increment term
    c = 321     

    # Number of Random numbers
    # to be generated
    number_observations = 100

    # To store random numbers
    randomNum = [0] * (number_observations)

    # Function Call
    linearCongruentialMethod(Xo, m, a, c,
                            randomNum,
                            number_observations)

    # Print the generated random numbers
    for i in randomNum:      
        print(i, end = ",")
#Normalizing the random numbers        
randomNu=[i/m for i in randomNum] 
randomNums=randomNu[1:]


print ("---------CHI-SQ_TEST-----------")
data_points = divide_RNG_data_into_10_equal_subdivisions_and_count(randomNums)
chi_sq_result = chi_square_uniformity_test(data_points, 0, number_observations)
chi_sq_significance_test( chi_sq_result, 0.05 )
chi_sq_significance_test( chi_sq_result, 0.8 )
chi_sq_significance_test( chi_sq_result, 0.9 )
chi_sq_significance_test( chi_sq_result, 0.95 )

print ("---------KS_TEST-----------")
first_100_values = collect_first_100_samples_in_data_set(randomNums)
first_100_values.sort()
ks_result = kolmogorov_smirnov_test(first_100_values,1,100)
ks_significance_test(ks_result,100, 0.1)
ks_significance_test(ks_result,100, 0.05)
ks_significance_test(ks_result,100, 0.01)
print ("Kolmogorov-Smirnov Test Result for D-Value: " + str(ks_result))
    


123456789.0,22640.0,57233.0,55076.0,36205.0,51704.0,56745.0,38380.0,61253.0,26624.0,47425.0,30004.0,51869.0,14984.0,2649.0,48380.0,30581.0,51600.0,59633.0,13636.0,52685.0,7448.0,60681.0,9740.0,9637.0,48416.0,11937.0,22356.0,22269.0,45480.0,17849.0,4380.0,47573.0,33456.0,19025.0,7012.0,9773.0,14392.0,54377.0,48684.0,62469.0,23104.0,64513.0,49524.0,64349.0,61640.0,22809.0,27964.0,37941.0,33744.0,945.0,35204.0,38541.0,7000.0,37833.0,24140.0,23141.0,16224.0,8545.0,45972.0,47037.0,63464.0,17529.0,53596.0,1685.0,52464.0,5393.0,32676.0,7917.0,50808.0,11049.0,1644.0,22725.0,27776.0,40641.0,11700.0,35869.0,50952.0,2009.0,15740.0,4341.0,24080.0,32369.0,64964.0,48973.0,14744.0,39561.0,46732.0,61221.0,57760.0,29729.0,12244.0,30845.0,24104.0,41785.0,45468.0,45909.0,14128.0,16337.0,996.0,---------CHI-SQ_TEST-----------
Print Significance Level: 0.05
Chi Sq: 7.300000000000001
Crit Value: 9768.5251
Result is: FAIL TO REJECT null hypothesis
....................................
Print Significance Level:

# BLUM BLUM SHUB

In [12]:

import random;
import sys;
import math
import statistics
import numpy as np
def divide_RNG_data_into_10_equal_subdivisions_and_count( data_points ):
    subdivisions = {  "1":  0,
                      "2":  0,
                      "3":  0,
                      "4":  0,
                      "5":  0,
                      "6":  0,
                      "7":  0,
                      "8":  0,
                      "9":  0,
                      "10": 0   }
 

    # Loop through our data points and count number of data points in each subdivision
    # Divide by tenths, from 0.0 to 1.0.
    for num in data_points:
        num = float(num)
        if num < 0.1:
            subdivisions["1"] += 1
        elif num < 0.2:
            subdivisions["2"] += 1
        elif num < 0.3:
            subdivisions["3"] += 1
        elif num < 0.4:
            subdivisions["4"] += 1
        elif num < 0.5:
            subdivisions["5"] += 1
        elif num < 0.6:
            subdivisions["6"] += 1
        elif num < 0.7:
            subdivisions["7"] += 1
        elif num < 0.8:
            subdivisions["8"] += 1
        elif num < 0.9:
            subdivisions["9"] += 1
        elif num < 1.0:
            subdivisions["10"] += 1

    return subdivisions

def chi_sq_significance_test( chi_sq, signif_level):
    result = "FAIL TO REJECT null hypothesis"
    crit_value = 0.0
    if signif_level == 0.8:
        crit_value = 10118.8246
    elif signif_level == 0.90:
        crit_value = 10181.6616
    elif signif_level == 0.95:
        crit_value = 10233.7489
    elif signif_level == 0.05:    
        crit_value = 9768.5251
    else:
        print( "**Invalid Significance Level for Chi Sq***")

    if chi_sq > crit_value:
        result = "REJECT null hypothesis"

    print ("Print Significance Level: " + str(signif_level))
    print ("Chi Sq: " + str(chi_sq))
    print ("Crit Value: " + str(crit_value))
    print ("Result is: " + result)
    print ("....................................")

    return result

def chi_square_uniformity_test( data_set, confidence_level, num_samples ):
    chi_sq_value = 0.0
    degrees_of_freedom = num_samples - 1

    # We're doing 10 equal subdivisions, so need to divide our number samples by 10,
    # Assuming uniform distribution, to get an expected value. All values should be same
    # If our distro is actually uniform.
    expected_val = num_samples/10.0


    # Loop through a dictionary and get every count
    # The observed value is going to be our count at each key, and then we can do chi-square
    for observed_val in data_set:
        # print "Observed value is: " + observed_val
        chi_sq_value += ( pow((expected_val - data_set[observed_val]), 2)/expected_val )

    # Coming out of this loop, we'll have a chi-squared test statistic
    # Now we just need to do a lookup to see if it's valid
    return chi_sq_value

def kolmogorov_smirnov_test( data_set, confidence_level, num_samples ):
    
    # Step 1:  Rank data from smallest to largest, such that:
    # R(1) <= R(2) <= R(3) ... <= R(i)
    data_set.sort()

    # Step 2: Computer D+ and D-
    # D+ = max(i/N - R(i))
    d_plus = get_d_plus_value_for_KS_TEST(data_set, num_samples)
    print ("D+ VALUE ="+str(d_plus))

    # D- = max(R(i) - (i -1)/n)
    d_minus = get_d_minus_value_for_KS_TEST(data_set, num_samples)
    print ("D- VALUE="+str(d_minus))

    # Step 3:  Computer D = max(D+,D-)
    d_value = max(d_plus, d_minus)
    print ("D VALUE (max): "+str(d_value))

    # Step 4: Determine critical value, using table
    # Step 5: Accept or reject Null hypothesis
    return d_value
def collect_first_100_samples_in_data_set( data_file ):
   

    first_100_vals_as_FLOATS = data_file[:100]
    #print("f=-",first_100_vals_as_FLOATS)
    return first_100_vals_as_FLOATS   
def ks_significance_test( d_statistic, num_observations, alpha_level ):
    """
    Perform Significance test for Kolmogorov-Smirnov
    Uses formulas from table A.7:  Discrete-Event System Simulation, by Banks and Carson, 1984
    :param d_statistic: The d-value we are testing
    :param num_observations: The number of observations in our data set
    :param alpha_level: The level of significance we are testing
    :return: result -- accept or reject
    """
    result = "FAIL TO REJECT null hypothesis"
    critical_value = 0


    if alpha_level == 0.1:
        critical_value = 1.22/np.sqrt(num_observations)
    elif alpha_level == 0.05:
        critical_value = 1.36/np.sqrt(num_observations)
    elif alpha_level == 0.01:
        critical_value = 1.63/np.sqrt(num_observations)
    else:
        print ("Invalid alpha level for KS test. Must be: 0.1, 0.05, or 0.01")

    if d_statistic > critical_value:
        result = "REJECT null hypothesis"
    print ("Alpha Level is: " + str(alpha_level))
    print ("D_statistic is: " + str(d_statistic))
    print ("Critical value is: " + str(critical_value))
    print ("Result is: " + result)
    print ("............................")

    return result
def get_d_plus_value_for_KS_TEST( data_set, num_samples ):
    """
    Finds the D+ value for a KS test
    :param data_set: 100 values, must be a list of floats
    :return: the D-+Statistic for our data set
    """
    # D+ = max(i/N - R(i))
    d_plus_max = 0
    value_rank_i = 1

    # iterate through data set
    for value in data_set:
        # Do each D+ calculation, store it
        d_plus_i_value = ( (value_rank_i/num_samples) - value )

        # Check if it is highest D+ value yet
        if d_plus_i_value > d_plus_max:
            d_plus_max = d_plus_i_value

        # increment our "i" value
        value_rank_i = value_rank_i + 1

    # coming out of this loop, D+ = highest D+ value
    return d_plus_max


def get_d_minus_value_for_KS_TEST( data_set, num_samples ):
    """
    Finds the D- value for a KS test
    :param data_set: 100 values, must be a list of floats
    :return: the D- Statistic for our data set
    """
    # D- = max(R(i) - (i -1)/n)
    d_minus_max = 0
    value_rank_i = 1.0

    # iterate through data set
    for value in data_set:
        # Do each D+ calculation, store it
        substraction_value = ( (value_rank_i - 1.0)/num_samples )
        d_minus_i_value = value - substraction_value

        # Check if it is highest D+ value yet
        if d_minus_i_value > d_minus_max:
            d_minus_max = d_minus_i_value

        # increment our "i" value
        value_rank_i = value_rank_i + 1

    # coming out of this loop, D+ = highest D+ value
    return d_minus_max


def bigppr(bits=256):
    """
    Randomly generate a probable prime with a given
    number of hex digits
    """
      
    candidate = random.getrandbits(bits)
 
    if candidate&1==0:
        candidate += 1
    prob = 0
    while 1:
        prob=pptest(candidate)
        if prob>0: break
        else: candidate += 2
    return candidate
         
def pptest(n):
    """
    Simple implementation of Miller-Rabin test for
    determining probable primehood.
    """
    bases  = [random.randrange(2,1000) for x in range(90)]
 
    # if any of the primes is a factor, we're done
 
    if n<=1: return 0
     
    for b in bases:
        if n%b==0: return 0
         
    tests,s  = 0,0
    m        = n-1
 
    # turning (n-1) into (2**s) * m
 
    while not m&1:  # while m is even
 
        m >>= 1
        s += 1
    for b in bases:
        tests += 1
        isprob = algP(m,s,b,n)
        if not isprob: break
             
    if isprob: return (1-(1./(4**tests)))
    else:      return 0
 
def algP(m,s,b,n):
    """
    based on Algorithm P in Donald Knuth's 'Art of
    Computer Programming' v.2 pg. 395 
    """
    result = 0
    y = pow(b,m,n)    
    for j in range(s):
       if (y==1 and j==0) or (y==n-1):
          result = 1
          break
       y = pow(y,2,n)       
    return result
 
class BlumBlumShub(object):
 
 
    def getPrime(self, bits):
        """
        Generate appropriate prime number for use in Blum-Blum-Shub.
          
        This generates the appropriate primes (p = 3 mod 4) needed to compute the
        "n-value" for Blum-Blum-Shub.
          
        bits - Number of bits in prime
 
        """
        bits=int(bits)
        while True:
            p = bigppr(bits);
            if p%4 == 3:
                break
        return p
 
    def generateN(self, bits):
        """
        This generates the "n value" for use in the Blum-Blum-Shub algorithm.
        
        bits - The number of bits of security
        """
     
        p = self.getPrime(bits/2)
        q = self.getPrime(bits/2)
 
        # make sure p != q (almost always true, but just in case, check)
        while p == q:
            q = getPrime(self, bits/2);
         
        # print "GenerateN - p = " + repr(p) + ", q = " + repr(q)            
        return p * q;
     
 
    def __init__(self, bits):
        """
        Constructor, specifing bits for n.
          
        bits - number of bits
        """       
        self.n = self.generateN(bits)
        #print ("n set to " + repr(self.n))
        length = self.bitLen(self.n)
        seed = random.getrandbits(length)
        self.setSeed(seed)  
 
    def setSeed(self, seed):
        """
        Sets or resets the seed value and internal state.
          
        seed -The new seed
        """
     
        self.state = seed % self.n
     
    def bitLen(self, x):
        " Get the bit lengh of a positive number"
        assert(x>0)
        q = 0
        while x: 
            q = q+1
            x = x>>1
        return q     
 
    def next(self, numBits):
        "Returns up to numBit random bits"
         
        result = 0
        for i in range(numBits,0,-1):
            self.state = (self.state**2) % self.n
            result = (result << 1) | (self.state&1)
         
        return result    
 
def main():
     
    bbs = BlumBlumShub(128);
         
    #print "Generating 10 numbers"
     
    print ("type: u")
    print ("numbit: 10")
    print ("count: 1000")
    number_observations=100
    rannums=[]
    for i in range (0,100):
        rannums.append(bbs.next(10))
        print (rannums[i],end=",")
    randomNu=[i/1024 for i in rannums] 
    randomNums=randomNu[1:]    
    print ("---------CHI-SQ_TEST-----------")
    data_points = divide_RNG_data_into_10_equal_subdivisions_and_count(randomNums)
    chi_sq_result = chi_square_uniformity_test(data_points, 0, number_observations)
    chi_sq_significance_test( chi_sq_result, 0.05 )
    chi_sq_significance_test( chi_sq_result, 0.8 )
    chi_sq_significance_test( chi_sq_result, 0.9 )
    chi_sq_significance_test( chi_sq_result, 0.95 )

    print ("---------KS_TEST-----------")
    first_100_values = collect_first_100_samples_in_data_set(randomNums)
    first_100_values.sort()
    ks_result = kolmogorov_smirnov_test(first_100_values,1,100)
    ks_significance_test(ks_result,100, 0.1)
    ks_significance_test(ks_result,100, 0.05)
    ks_significance_test(ks_result,100, 0.01)
    print ("Kolmogorov-Smirnov Test Result for D-Value: " + str(ks_result))    
             
if __name__ == "__main__":
    main()

type: u
numbit: 10
count: 1000
615,411,928,828,280,171,943,623,557,695,309,891,1009,138,501,770,232,202,106,718,115,114,932,331,289,402,637,431,814,351,835,403,671,705,743,799,332,430,632,688,159,131,590,908,366,293,668,493,991,536,204,933,560,220,65,1018,729,84,534,132,167,984,678,499,819,568,43,365,165,232,311,627,610,99,658,252,76,34,690,160,747,432,340,530,127,948,806,25,78,901,140,1004,202,719,893,181,885,582,881,598,---------CHI-SQ_TEST-----------
Print Significance Level: 0.05
Chi Sq: 7.900000000000002
Crit Value: 9768.5251
Result is: FAIL TO REJECT null hypothesis
....................................
Print Significance Level: 0.8
Chi Sq: 7.900000000000002
Crit Value: 10118.8246
Result is: FAIL TO REJECT null hypothesis
....................................
Print Significance Level: 0.9
Chi Sq: 7.900000000000002
Crit Value: 10181.6616
Result is: FAIL TO REJECT null hypothesis
....................................
Print Significance Level: 0.95
Chi Sq: 7.900000000000002
Crit Value: